# 지도학습 - 회귀

## 1) 라이브러리 및 데이터 불러오기

In [1]:
%pip install statsmodels

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB 1.9 MB/s eta 0:00:05
   ------ --------------------------------- 1.5/9.6 MB 23.9 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.6 MB 27.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.6 MB 33.5 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.6 MB 37.3 MB/s eta 0:00:01
   ----------------------------------- ---- 8.6/9.6 MB 34.2 MB/s eta 0:00:01
   ------------------------------------ --- 8.6/9.6 MB 30.7 MB/s eta 0:00:01
   ------------------------------------ --- 8.8/9.6 MB 25.6 MB/s eta 0:00:01
   ------------------------------------- -- 9.0/9.6 MB 23.0 MB/s eta 0:00:01
   -------------------------------------- - 9.3/9.6 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 19.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/233.3 kB ? eta -:--:--
   -----------


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, recall_score, f1_score, classification_report
)
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

In [5]:
"""
스쿨존 안전점수 다중선형회귀 분석 파이프라인
Step 1 : 데이터 로드 & 기본 탐색
Step 2 : 상관관계 분석
Step 3 : 다중공선성 검사 (VIF)
Step 4 : 모델 학습 (Train/Test 80:20)
Step 5 : 회귀 지표 (R², MAE, RMSE)
Step 6 : 분류 지표 (Accuracy, Recall, F1 — 3구간 이산화)
Step 7 : 잔차 분석 (정규성, 등분산성)
Step 8 : 시각화 종합
"""

# ═══════════════════════════════════════════════════════════════════
# STEP 1 ── 데이터 로드 & 기본 탐색
# ═══════════════════════════════════════════════════════════════════
df = pd.read_csv('./data/스쿨존_안전점수.csv', encoding='utf-8-sig')

print("=" * 60)
print("STEP 1 | 데이터 기본 탐색")
print("=" * 60)
print(f"데이터 크기: {df.shape[0]}행 × {df.shape[1]}열")
print(f"\n결측치:\n{df.isnull().sum()[df.isnull().sum() > 0].to_string() or '없음'}")
print(f"\n기술통계:\n{df.describe().round(3).to_string()}")

X_raw = df.drop(columns=DROP)
y     = df[LABEL]
features = X_raw.columns.tolist()
print(f"\n사용 피처 ({len(features)}개): {features}")
print(f"레이블 분포 — min:{y.min():.1f}  max:{y.max():.1f}  mean:{y.mean():.2f}  std:{y.std():.2f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 2 ── 상관관계 분석
# 피처와 레이블 간 피어슨 상관계수를 계산해
# 어떤 변수가 안전점수에 강하게 연관되는지 파악
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 2 | 상관관계 분석 (피어슨 r)")
print("=" * 60)

corr_with_label = X_raw.apply(lambda col: col.corr(y))
corr_df = corr_with_label.sort_values(ascending=False).to_frame(name='r')
corr_df['|r|'] = corr_df['r'].abs()
print(corr_df.round(4).to_string())

# ═══════════════════════════════════════════════════════════════════
# STEP 3 ── 다중공선성 검사 (VIF)
# VIF > 10 이면 해당 피처가 다른 피처들과 강하게 선형 관련된다는 신호
# → 회귀계수 해석이 불안정해질 수 있음
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 3 | 다중공선성 검사 (VIF)")
print("=" * 60)

scaler_vif = StandardScaler()
X_scaled_vif = scaler_vif.fit_transform(X_raw)

vif_data = pd.DataFrame({
    'Feature': features,
    'VIF': [variance_inflation_factor(X_scaled_vif, i)
            for i in range(X_scaled_vif.shape[1])]
}).sort_values('VIF', ascending=False)

print(vif_data.round(2).to_string(index=False))
high_vif = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
if high_vif:
    print(f"\n⚠️  VIF > 10 (다중공선성 의심): {high_vif}")
else:
    print("\n✅ 심각한 다중공선성 없음 (모두 VIF ≤ 10)")

# ═══════════════════════════════════════════════════════════════════
# STEP 4 ── 모델 학습 (Train 80 / Test 20)
# StandardScaler: 스케일이 다른 피처들을 평균 0, 분산 1로 정규화
# → 회귀계수 크기를 피처 간 공정하게 비교 가능
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 4 | 모델 학습")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_s, y_train)

y_pred_train = model.predict(X_train_s)
y_pred_test  = model.predict(X_test_s)

print(f"Train 샘플: {len(X_train)}  |  Test 샘플: {len(X_test)}")
print(f"절편 (intercept): {model.intercept_:.4f}")

coef_df = pd.DataFrame({
    'Feature': features,
    'Coef': model.coef_
}).sort_values('Coef', ascending=False)
print(f"\n회귀계수:\n{coef_df.round(4).to_string(index=False)}")

# ═══════════════════════════════════════════════════════════════════
# STEP 5 ── 회귀 지표
# R²: 1에 가까울수록 설명력 좋음
# MAE: 예측 오차의 평균 절댓값
# RMSE: 이상치에 민감한 오차 지표
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 5 | 회귀 지표")
print("=" * 60)

train_r2  = r2_score(y_train, y_pred_train)
test_r2   = r2_score(y_test,  y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae  = mean_absolute_error(y_test,  y_pred_test)
train_rmse= np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test,  y_pred_test))

metrics = pd.DataFrame({
    'Metric': ['R²', 'MAE', 'RMSE'],
    'Train':  [train_r2,   train_mae,  train_rmse],
    'Test':   [test_r2,    test_mae,   test_rmse],
})
print(metrics.round(4).to_string(index=False))
print(f"\n과적합 지수 (Train R² - Test R²): {(train_r2 - test_r2):.4f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 6 ── 분류 지표 (3구간 이산화)
# 회귀값을 저(0~50) / 중(50~75) / 고(75~) 구간으로 변환해
# Accuracy, Recall, F1-score를 계산 — 분류 성능 관점 추가 해석
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 6 | 분류 지표 (3구간: 저·중·고)")
print("=" * 60)

BINS   = [float('-inf'), 50, 75, float('inf')]
LABELS = [0, 1, 2]
CLASS_NAMES = ['저(~50)', '중(50~75)', '고(75~)']

def to_cls(s):
    return pd.cut(pd.Series(s).reset_index(drop=True),
                  bins=BINS, labels=LABELS).astype(int).values

y_test_cls = to_cls(y_test)
y_pred_cls = to_cls(y_pred_test)

print(f"Accuracy : {accuracy_score(y_test_cls, y_pred_cls):.4f}")
print(f"Recall   : {recall_score(y_test_cls, y_pred_cls, average='macro', zero_division=0):.4f}")
print(f"F1-Score : {f1_score(y_test_cls, y_pred_cls, average='macro', zero_division=0):.4f}")
print(f"\n분류 리포트:\n"
      f"{classification_report(y_test_cls, y_pred_cls, target_names=CLASS_NAMES, zero_division=0)}")

# ═══════════════════════════════════════════════════════════════════
# STEP 7 ── 잔차 분석
# 잔차(residual) = 실제값 - 예측값
# 정규성: Shapiro-Wilk 검정 — p > 0.05 이면 정규분포 가정 만족
# 등분산성: 잔차가 예측값에 따라 패턴 없이 고르게 퍼져야 함
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 7 | 잔차 분석")
print("=" * 60)

residuals = y_test.values - y_pred_test
stat, p_value = stats.shapiro(residuals)
print(f"Shapiro-Wilk 검정 — W={stat:.4f}, p={p_value:.4f}")
print("→ 잔차 정규성:", "✅ 만족 (p > 0.05)" if p_value > 0.05 else "⚠️ 불만족 (p ≤ 0.05)")
print(f"잔차 평균: {residuals.mean():.4f}  |  잔차 표준편차: {residuals.std():.4f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 8 ── 시각화 (6개 패널)
# ═══════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
fig.suptitle('스쿨존 안전점수 다중선형회귀 분석 결과', fontsize=16, fontweight='bold', y=1.01)

# (1) 훈련 vs 테스트 R² 비교 막대
ax = axes[0, 0]
bars = ax.bar(['Train R²', 'Test R²'], [train_r2, test_r2],
               color=['steelblue', 'coral'], width=0.4, edgecolor='white')
ax.set_ylim(0, 1.15)
ax.set_ylabel('R² Score')
ax.set_title('① 훈련 vs 테스트 R²')
ax.axhline(1.0, color='gray', linestyle='--', linewidth=0.8)
for bar, val in zip(bars, [train_r2, test_r2]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.4f}', ha='center', fontsize=12, fontweight='bold')

# (2) 실제값 vs 예측값 산점도
ax = axes[0, 1]
ax.scatter(y_test, y_pred_test, alpha=0.8, color='steelblue', edgecolors='white', s=70)
mn = min(y_test.min(), y_pred_test.min()) - 2
mx = max(y_test.max(), y_pred_test.max()) + 2
ax.plot([mn, mx], [mn, mx], 'r--', label='완벽 예측선')
ax.set_xlabel('실제 안전점수')
ax.set_ylabel('예측 안전점수')
ax.set_title('② 실제값 vs 예측값')
ax.legend()

# (3) 피처별 회귀계수
ax = axes[0, 2]
colors = ['coral' if c < 0 else 'steelblue' for c in coef_df['Coef']]
ax.barh(coef_df['Feature'], coef_df['Coef'], color=colors, edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('회귀계수')
ax.set_title('③ 피처별 회귀계수')

# (4) 상관계수 막대
ax = axes[1, 0]
sorted_corr = corr_df.sort_values('r')
ax.barh(sorted_corr.index, sorted_corr['r'],
        color=['coral' if v < 0 else 'steelblue' for v in sorted_corr['r']],
        edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('피어슨 r')
ax.set_title('④ 레이블과의 상관계수')

# (5) 잔차 분포 (히스토그램 + 정규분포 곡선)
ax = axes[1, 1]
ax.hist(residuals, bins=12, color='steelblue', edgecolor='white', density=True, alpha=0.8)
xmin, xmax = ax.get_xlim()
x_range = np.linspace(xmin, xmax, 100)
ax.plot(x_range, stats.norm.pdf(x_range, residuals.mean(), residuals.std()),
        'r-', linewidth=2, label='정규분포 곡선')
ax.set_xlabel('잔차')
ax.set_ylabel('밀도')
ax.set_title('⑤ 잔차 분포')
ax.legend()

# (6) 잔차 vs 예측값 (등분산성 확인)
ax = axes[1, 2]
ax.scatter(y_pred_test, residuals, alpha=0.8, color='steelblue', edgecolors='white', s=70)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_xlabel('예측값')
ax.set_ylabel('잔차')
ax.set_title('⑥ 잔차 vs 예측값 (등분산성)')

plt.tight_layout()
plt.savefig('./outputs/regression_full_analysis.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n✅ 시각화 저장 완료: regression_full_analysis.png")

STEP 1 | 데이터 기본 탐색
데이터 크기: 60행 × 16열

결측치:
Series([], )

기술통계:
           위도       경도  도로안전표지  도로적색표면  무단횡단방지펜스  무인교통단속카메라  보호구역표지판  생활안전CCTV     신호등    옐로카펫    횡단보도     발생건수   어린이비율     안전점수
count  60.000   60.000  60.000  60.000    60.000     60.000   60.000    60.000  60.000  60.000  60.000   60.000  60.000   60.000
mean   37.425  127.141   4.250  10.450    17.767      2.850   16.567    24.517   5.967   1.117   4.333   13.917   0.086   65.242
std     0.031    0.016   2.614   6.498     9.407      2.082   10.429    13.884   3.425   0.940   2.412   20.602   0.034   17.391
min    37.338  127.112   0.000   1.000     0.000      0.000    3.000     4.000   0.000   0.000   0.000    1.000   0.035    0.000
25%    37.408  127.127   2.000   5.750    11.000      1.000   11.750    13.750   4.000   1.000   2.750    4.000   0.055   57.200
50%    37.439  127.140   4.000   8.500    15.500      2.000   14.500    23.000   5.000   1.000   4.000    7.000   0.080   66.375
75%    37.448  127.155   6.000  15

# 발생건수를 레이블로

In [10]:
"""
스쿨존 안전점수 다중선형회귀 분석 파이프라인
Step 1 : 데이터 로드 & 기본 탐색
Step 2 : 상관관계 분석
Step 3 : 다중공선성 검사 (VIF)
Step 4 : 모델 학습 (Train/Test 80:20)
Step 5 : 회귀 지표 (R², MAE, RMSE)
Step 6 : 분류 지표 (Accuracy, Recall, F1 — 3구간 이산화)
Step 7 : 잔차 분석 (정규성, 등분산성)
Step 8 : 시각화 종합
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, recall_score, f1_score, classification_report
)
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

# ── 한글 폰트 ─────────────────────────────────────────────────────
plt.rcParams['axes.unicode_minus'] = False
font_list = [f.name for f in fm.fontManager.ttflist
             if any(k in f.name for k in ['Nanum', 'Gothic', 'Malgun'])]
plt.rcParams['font.family'] = font_list[0] if font_list else 'DejaVu Sans'

LABEL = '발생건수'
DROP  = ['시설물명', '행정동', '위도', '경도', '안전점수', LABEL]

# ═══════════════════════════════════════════════════════════════════
# STEP 1 ── 데이터 로드 & 기본 탐색
# ═══════════════════════════════════════════════════════════════════
df = pd.read_csv('./data/스쿨존_안전점수.csv', encoding='utf-8-sig')

print("=" * 60)
print("STEP 1 | 데이터 기본 탐색")
print("=" * 60)
print(f"데이터 크기: {df.shape[0]}행 × {df.shape[1]}열")
print(f"\n결측치:\n{df.isnull().sum()[df.isnull().sum() > 0].to_string() or '없음'}")
print(f"\n기술통계:\n{df.describe().round(3).to_string()}")

X_raw = df.drop(columns=DROP)
y     = df[LABEL]
features = X_raw.columns.tolist()
print(f"\n사용 피처 ({len(features)}개): {features}")
print(f"레이블 분포 — min:{y.min():.1f}  max:{y.max():.1f}  mean:{y.mean():.2f}  std:{y.std():.2f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 2 ── 상관관계 분석
# 피처와 레이블 간 피어슨 상관계수를 계산해
# 어떤 변수가 안전점수에 강하게 연관되는지 파악
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 2 | 상관관계 분석 (피어슨 r)")
print("=" * 60)

corr_with_label = X_raw.apply(lambda col: col.corr(y))
corr_df = corr_with_label.sort_values(ascending=False).to_frame(name='r')
corr_df['|r|'] = corr_df['r'].abs()
print(corr_df.round(4).to_string())

# ═══════════════════════════════════════════════════════════════════
# STEP 3 ── 다중공선성 검사 (VIF)
# VIF > 10 이면 해당 피처가 다른 피처들과 강하게 선형 관련된다는 신호
# → 회귀계수 해석이 불안정해질 수 있음
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 3 | 다중공선성 검사 (VIF)")
print("=" * 60)

scaler_vif = StandardScaler()
X_scaled_vif = scaler_vif.fit_transform(X_raw)

vif_data = pd.DataFrame({
    'Feature': features,
    'VIF': [variance_inflation_factor(X_scaled_vif, i)
            for i in range(X_scaled_vif.shape[1])]
}).sort_values('VIF', ascending=False)

print(vif_data.round(2).to_string(index=False))
high_vif = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
if high_vif:
    print(f"\n⚠️  VIF > 10 (다중공선성 의심): {high_vif}")
else:
    print("\n✅ 심각한 다중공선성 없음 (모두 VIF ≤ 10)")

# ═══════════════════════════════════════════════════════════════════
# STEP 4 ── 모델 학습 (Train 80 / Test 20)
# StandardScaler: 스케일이 다른 피처들을 평균 0, 분산 1로 정규화
# → 회귀계수 크기를 피처 간 공정하게 비교 가능
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 4 | 모델 학습")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_s, y_train)

y_pred_train = model.predict(X_train_s)
y_pred_test  = model.predict(X_test_s)

print(f"Train 샘플: {len(X_train)}  |  Test 샘플: {len(X_test)}")
print(f"절편 (intercept): {model.intercept_:.4f}")

coef_df = pd.DataFrame({
    'Feature': features,
    'Coef': model.coef_
}).sort_values('Coef', ascending=False)
print(f"\n회귀계수:\n{coef_df.round(4).to_string(index=False)}")

# ═══════════════════════════════════════════════════════════════════
# STEP 5 ── 회귀 지표
# R²: 1에 가까울수록 설명력 좋음
# MAE: 예측 오차의 평균 절댓값
# RMSE: 이상치에 민감한 오차 지표
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 5 | 회귀 지표")
print("=" * 60)

train_r2  = r2_score(y_train, y_pred_train)
test_r2   = r2_score(y_test,  y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae  = mean_absolute_error(y_test,  y_pred_test)
train_rmse= np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test,  y_pred_test))

metrics = pd.DataFrame({
    'Metric': ['R²', 'MAE', 'RMSE'],
    'Train':  [train_r2,   train_mae,  train_rmse],
    'Test':   [test_r2,    test_mae,   test_rmse],
})
print(metrics.round(4).to_string(index=False))
print(f"\n과적합 지수 (Train R² - Test R²): {(train_r2 - test_r2):.4f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 6 ── 분류 지표 (3구간 이산화)
# 회귀값을 저(0~50) / 중(50~75) / 고(75~) 구간으로 변환해
# Accuracy, Recall, F1-score를 계산 — 분류 성능 관점 추가 해석
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 6 | 분류 지표 (3구간: 저·중·고)")
print("=" * 60)

BINS   = [float('-inf'), 5, 20, float('inf')]
LABELS = [0, 1, 2]
CLASS_NAMES = ['저(~5건)', '중(5~20건)', '고(20건~)']

def to_cls(s):
    return pd.cut(pd.Series(s).reset_index(drop=True),
                  bins=BINS, labels=LABELS).astype(int).values

y_test_cls = to_cls(y_test)
y_pred_cls = to_cls(y_pred_test)

print(f"Accuracy : {accuracy_score(y_test_cls, y_pred_cls):.4f}")
print(f"Recall   : {recall_score(y_test_cls, y_pred_cls, average='macro', zero_division=0):.4f}")
print(f"F1-Score : {f1_score(y_test_cls, y_pred_cls, average='macro', zero_division=0):.4f}")
print(f"\n분류 리포트:\n"
      f"{classification_report(y_test_cls, y_pred_cls, target_names=CLASS_NAMES, zero_division=0)}")

# ═══════════════════════════════════════════════════════════════════
# STEP 7 ── 잔차 분석
# 잔차(residual) = 실제값 - 예측값
# 정규성: Shapiro-Wilk 검정 — p > 0.05 이면 정규분포 가정 만족
# 등분산성: 잔차가 예측값에 따라 패턴 없이 고르게 퍼져야 함
# ═══════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("STEP 7 | 잔차 분석")
print("=" * 60)

residuals = y_test.values - y_pred_test
stat, p_value = stats.shapiro(residuals)
print(f"Shapiro-Wilk 검정 — W={stat:.4f}, p={p_value:.4f}")
print("→ 잔차 정규성:", "✅ 만족 (p > 0.05)" if p_value > 0.05 else "⚠️ 불만족 (p ≤ 0.05)")
print(f"잔차 평균: {residuals.mean():.4f}  |  잔차 표준편차: {residuals.std():.4f}")

# ═══════════════════════════════════════════════════════════════════
# STEP 8 ── 시각화 (6개 패널)
# ═══════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
fig.suptitle('스쿨존 발생건수 다중선형회귀 분석 결과', fontsize=16, fontweight='bold', y=1.01)

# (1) 훈련 vs 테스트 R² 비교 막대
ax = axes[0, 0]
bars = ax.bar(['Train R²', 'Test R²'], [train_r2, test_r2],
               color=['steelblue', 'coral'], width=0.4, edgecolor='white')
ax.set_ylim(0, 1.15)
ax.set_ylabel('R² Score')
ax.set_title('① 훈련 vs 테스트 R²')
ax.axhline(1.0, color='gray', linestyle='--', linewidth=0.8)
for bar, val in zip(bars, [train_r2, test_r2]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.4f}', ha='center', fontsize=12, fontweight='bold')

# (2) 실제값 vs 예측값 산점도
ax = axes[0, 1]
ax.scatter(y_test, y_pred_test, alpha=0.8, color='steelblue', edgecolors='white', s=70)
mn = min(y_test.min(), y_pred_test.min()) - 2
mx = max(y_test.max(), y_pred_test.max()) + 2
ax.plot([mn, mx], [mn, mx], 'r--', label='완벽 예측선')
ax.set_xlabel('실제 안전점수')
ax.set_ylabel('예측 안전점수')
ax.set_title('② 실제값 vs 예측값')
ax.legend()

# (3) 피처별 회귀계수
ax = axes[0, 2]
colors = ['coral' if c < 0 else 'steelblue' for c in coef_df['Coef']]
ax.barh(coef_df['Feature'], coef_df['Coef'], color=colors, edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('회귀계수')
ax.set_title('③ 피처별 회귀계수')

# (4) 상관계수 막대
ax = axes[1, 0]
sorted_corr = corr_df.sort_values('r')
ax.barh(sorted_corr.index, sorted_corr['r'],
        color=['coral' if v < 0 else 'steelblue' for v in sorted_corr['r']],
        edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('피어슨 r')
ax.set_title('④ 레이블과의 상관계수')

# (5) 잔차 분포 (히스토그램 + 정규분포 곡선)
ax = axes[1, 1]
ax.hist(residuals, bins=12, color='steelblue', edgecolor='white', density=True, alpha=0.8)
xmin, xmax = ax.get_xlim()
x_range = np.linspace(xmin, xmax, 100)
ax.plot(x_range, stats.norm.pdf(x_range, residuals.mean(), residuals.std()),
        'r-', linewidth=2, label='정규분포 곡선')
ax.set_xlabel('잔차')
ax.set_ylabel('밀도')
ax.set_title('⑤ 잔차 분포')
ax.legend()

# (6) 잔차 vs 예측값 (등분산성 확인)
ax = axes[1, 2]
ax.scatter(y_pred_test, residuals, alpha=0.8, color='steelblue', edgecolors='white', s=70)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_xlabel('예측값')
ax.set_ylabel('잔차')
ax.set_title('⑥ 잔차 vs 예측값 (등분산성)')

plt.tight_layout()
plt.savefig('./data/regression_accident_analysis_accident_label.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n✅ 시각화 저장 완료: regression_accident_analysis.png")

STEP 1 | 데이터 기본 탐색
데이터 크기: 60행 × 16열

결측치:
Series([], )

기술통계:
           위도       경도  도로안전표지  도로적색표면  무단횡단방지펜스  무인교통단속카메라  보호구역표지판  생활안전CCTV     신호등    옐로카펫    횡단보도     발생건수   어린이비율     안전점수
count  60.000   60.000  60.000  60.000    60.000     60.000   60.000    60.000  60.000  60.000  60.000   60.000  60.000   60.000
mean   37.425  127.141   4.250  10.450    17.767      2.850   16.567    24.517   5.967   1.117   4.333   13.917   0.086   65.242
std     0.031    0.016   2.614   6.498     9.407      2.082   10.429    13.884   3.425   0.940   2.412   20.602   0.034   17.391
min    37.338  127.112   0.000   1.000     0.000      0.000    3.000     4.000   0.000   0.000   0.000    1.000   0.035    0.000
25%    37.408  127.127   2.000   5.750    11.000      1.000   11.750    13.750   4.000   1.000   2.750    4.000   0.055   57.200
50%    37.439  127.140   4.000   8.500    15.500      2.000   14.500    23.000   5.000   1.000   4.000    7.000   0.080   66.375
75%    37.448  127.155   6.000  15